In [ ]:
# Features-Extract.py
from __future__ import print_function
from keras.applications.inception_v3 import InceptionV3, conv2d_bn
from keras.applications.densenet  import DenseNet201
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import optimizers
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import h5py

conv_base = InceptionV3(weights='imagenet', include_top=False)
train_dir ='D:/OCT/dataset-size-299/train/'
validation_dir = 'D:/OCT/dataset-size-299/val/'

def extract_features(file_name, directory, key, sample_count, target_size, batch_size, class_mode='categorical'):
    h5_file = h5py.File(file_name, 'w')
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(directory, target_size=target_size,
        batch_size=batch_size, class_mode=class_mode)
    samples_processed = 0
    batch_number = 0
    if sample_count == 'all':
        sample_count = generator.n
    print_size = True
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        if print_size == True:
            print_size = False
            print('Features shape', features_batch.shape)
        samples_processed += inputs_batch.shape[0]
        h5_file.create_dataset('features-'+ str(batch_number), data=features_batch)
        h5_file.create_dataset('labels-'+str(batch_number), data=labels_batch)
        batch_number = batch_number + 1
        print("Batch:%d Sample:%d\r" % (batch_number,samples_processed), end="")
        if samples_processed >= sample_count:
            break
    h5_file.create_dataset('batches', data=batch_number)
    h5_file.close()
    return

extract_features('train.h5', train_dir, key='train', sample_count='all', batch_size=100, target_size=(299,299))

extract_features('val.h5', validation_dir, key='val', sample_count='all', batch_size=100, target_size=(299,299))


In [ ]:
# Features-Train.py
from __future__ import print_function
import keras
import tensorflow as tf
from keras import optimizers
from keras.applications.inception_v3 import conv2d_bn
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import h5py
import matplotlib.pyplot as plt

def features_from_file(path, ctx):
    h5f = h5py.File(path, 'r')
    batch_count = h5f['batches'][()]
    print(ctx, 'batches:', batch_count)       
    def generator():
        while True:
            for batch_id in range(0, batch_count):
                X = h5f['features-' + str(batch_id)]
                y = h5f['labels-' + str(batch_id)]
                yield X, y      
    return batch_count, generator()

train_steps_per_epoch, train_generator = features_from_file('D:/OCT/inceptionv3/train.h5', 'train')
validation_steps, validation_data = features_from_file('D:/OCT/inceptionv3/val.h5', 'val')
np.random.seed(7)
inputs = Input(shape=(8, 8, 2048))
x = conv2d_bn(inputs, 64, 1, 1)
x = Dropout(0.5)(x)
x = Flatten()(x)
outputs  = Dense(4, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

callbacks = [ ModelCheckpoint('D:/OCT/inceptionv3/model.features.{epoch:02d}-{val_acc:.2f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max', period=1),
              ReduceLROnPlateau(monitor='val_loss', verbose=1, factor=0.5, patience=5, min_lr=0.00005)]

history = model.fit_generator(generator=train_generator, steps_per_epoch=train_steps_per_epoch, validation_data=validation_data, validation_steps=validation_steps, epochs=100, callbacks=callbacks)

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    plt.figure(figsize=(12,8))
    plt.plot(epochs, acc, 'b', color='red' , label='Training Accuracy')
    plt.title('Training Accuracy')
    plt.legend()
    plt.figure(figsize=(12,8))
    plt.plot(epochs, val_acc, 'b', color='red', label='Validation Accuracy')
    plt.title('Validation Accuracy')
    plt.legend()
    plt.figure(figsize=(12,8))
    plt.plot(epochs, loss, 'b',color='red', label='Training Loss')
    plt.title('Training loss')
    plt.legend()
    plt.figure(figsize=(12,8))
    plt.plot(epochs, val_loss, 'b', color='red', label='Validation Loss')
    plt.title('Validation loss')
    plt.legend()
    plt.show()
    return acc, val_acc, loss, val_loss

acc, val_acc, loss, val_loss = plot_history(history)


In [ ]:
# Feature-Evaluate.py
import os
import numpy as np
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import load_model
from keras import backend as K
from io import BytesIO
from PIL import Image
import cv2
from tensorflow.keras import backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import colors
import requests

K.set_learning_phase(0) 
base_model = InceptionV3(weights='imagenet', include_top=False)
model = load_model('model.features.57-0.99.hdf5')
print(model.summary())

classes = ['CNV', 'DME', 'DRUSEN', 'NORMAL']
def preprocess_input(x):
    x = img_to_array(x) / 255.
    return np.expand_dims(x, axis=0) 

def predict_from_image_path(image_path):
    return predict_image(load_img(image_path, target_size=(299, 299)))

def predict_from_image_url(image_url):
    res = requests.get(image_url)
    im = Image.open(BytesIO(res.content))
    return predict_from_image_path(im.fp)
    
def predict_image(im):
    x = preprocess_input(im)
    x = base_model.predict(x)
    pred = np.argmax(model.predict(x))
    return pred, classes[pred]

def grad_CAM(image_path):
    im = load_img(image_path, target_size=(299,299))
    x = preprocess_input(im)
    x = base_model.predict(x)
    pred = model.predict(x)
    index = np.argmax(pred)
    class_output = model.output[:, index]
    last_conv_layer = model.get_layer('conv2d')
    nmb_channels = last_conv_layer.output.shape[3]
    grads = K.gradients(class_output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function(model.inputs, [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([x])
    for i in range(nmb_channels):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    img = cv2.imread(image_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.5 + img 
    plt.figure(figsize=(24,12))
    cv2.imwrite('./tmp.jpeg', superimposed_img)
    plt.imshow(mpimg.imread('./tmp.jpeg'))
    plt.title(image_path)
    plt.show()

for i, c in enumerate(classes):
    folder = 'D:/OCT/dataset-size-299/test/' + c + '/'
    count = 1
    for file in os.listdir(folder):
        if file.endswith('.jpeg') == True:
            image_path = folder + file
            p, class_name = predict_from_image_path(image_path)
            if p == i:
                print(file, p, class_name)
            else:
                print(file, p, class_name, '**INCORRECT PREDICTION**')
                grad_CAM(image_path)
        count = count +1
        if count == 100:
            continue